In [ ]:
def rejection(y_true, y_pred, b=0):
    return np.sum((1 - np.round(y_pred+b)) * y_true)

def acceptance(y_true, y_pred, b=0):
    return np.sum((1 - y_true) *  np.round(y_pred+b))

In [ ]:
rejection(x.label, x.pred), acceptance(x.label, x.pred)

In [ ]:
diff = rejection(x.label, x.pred)- acceptance(x.label, x.pred)
diff

In [ ]:
rejection(x.label, x.pred, b=b), acceptance(x.label, x.pred, b=b)

In [ ]:
delta = x[x.pred * np.sign(diff) <.5*np.sign(diff)]

delta['d'] = 0.5-delta.pred
s = delta.sort_values('d')
b = (s.iloc[int(diff)].d+s.iloc[int(diff)-1].d)/2
b

In [ ]:
delta.sort_values('d')

In [1]:
%load_ext autoreload
%autoreload 2
import re
import glob
p = re.compile('/([0-9]+)')
id_list = list({p.search(file).group(1) for file in glob.glob('./../Data/train_db/*')})
len(id_list)

194

In [2]:
id_list[4]

'1073'

In [ ]:
import tensorflow as tf
a = tf.ones(2)
b = tf.zeros(3)
s =tf.tile(a, [12])
s1 = tf.reshape(s, [12, -1])
sh = tf.shape(b)
b1, _ = tf.nn.top_k(b,2)
c= tf.concat([a,b1], axis=0)
with tf.Session() as sess:
  d = sess.run([b1, sh, s1])
d

In [ ]:
from lib.single_generator import SingleGenerator

In [ ]:
s, ls = sg.__getitem__(4)
ls

In [ ]:
from sklearn.model_selection import train_test_split
s1, s2 = train_test_split(ls, test_size=0.3)
len(s1), len(s2) 

In [3]:
import glob
file_list = [file for file in glob.glob('./../Data/train_db/*')]

len(file_list)

52284

In [4]:
from sklearn.model_selection import train_test_split
train_files, valid_files = train_test_split(file_list, test_size=0.1)
len(train_files), len(valid_files)

(47055, 5229)

In [ ]:
from lib.single_generator import SingleGenerator
sg = SingleGenerator(valid_files, id_list=id_list)

In [ ]:
s, ls = sg.__getitem__(4)
len(ls[0])

In [9]:
from lib.list_generator import ListGenerator
lg = ListGenerator(train_files, id_list, debug=True)

In [12]:
s, ls = lg.__getitem__(4)
ls[0]

First file = ./../Data/train_db/1007_09002.txt
Same file = ./../Data/train_db/1007_08006.txt
Another file = ./../Data/train_db/0071_04006.txt


1

In [11]:
len(s[0]), len(s[1])

(32, 32)

In [20]:
ls[31]

0

In [19]:
s[1][31]

array([ 1.11353e+00,  1.66791e+00, -8.57532e-01,  1.06652e+00,
       -1.02777e+00,  1.40157e+00, -7.03595e-02, -1.04474e+00,
        8.99240e-01, -1.11507e+00,  6.53107e-01,  2.13366e+00,
       -3.47951e-01,  5.69208e-01,  7.40644e-01, -7.42274e-02,
        4.45251e-01, -8.14747e-01, -6.12337e-01, -8.72586e-01,
       -1.80474e+00,  2.24388e-01,  1.86181e+00,  1.19594e+00,
       -1.28399e+00, -2.01186e-01, -1.96944e+00, -9.49091e-01,
       -1.74049e+00, -2.79279e+00,  1.56234e+00, -3.36224e+00,
        2.16378e+00, -1.09784e+00, -6.11149e-01,  2.24154e+00,
       -1.94467e-02, -3.12430e-01, -4.59901e-02, -5.96842e-01,
        1.93628e+00, -4.16477e-01, -4.85029e-01,  2.36134e+00,
        1.79863e+00, -2.22332e-01, -2.17524e+00,  1.56993e+00,
        2.49529e+00,  9.39548e-01, -7.82049e-01, -1.14475e+00,
        1.05417e+00,  1.02420e+00, -1.88752e-01, -1.49468e+00,
        6.99278e-02,  1.41610e+00, -1.13894e+00, -4.70572e-02,
       -1.66920e+00,  9.44267e-01,  7.69924e-01, -2.084

In [ ]:
from keras.models import Sequential, Model
from keras import regularizers
from keras.layers import Input, Dense, Activation, Dropout, BatchNormalization, Concatenate
a1 = Input(shape=(400,))
a2 = Input(shape=(400,))

resh = Concatenate(axis=-1)([a1, a2])
j = Dense(20)(resh)
i = BatchNormalization()(j)
k = Activation('relu')(i)
l = Dense(1)(k)
res = Activation('sigmoid')(l)
compiled_model = Model(inputs=[a1, a2], outputs=[res])

compiled_model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

compiled_history = compiled_model.fit_generator(generator=lg,
                    validation_data=lg,
                    epochs=1, steps_per_epoch=20, validation_steps=10 )